In [1]:
import numpy as np
import pandas as pd
import pickle
from IPython.core.display import HTML
import scipy.stats as stats
import statsmodels.stats.weightstats as wstats
import math

<div class="span5 alert alert-info">
<h2>Note book details</h2>

<p> This notebook is for <b>data story</b> for <b>Mortgage client segmentation</b> project.</p>

<p> Notes.</p>
<ol>
<li>  The file will statistical analysis to establish whether one field has a significant impact on the another</li> 
<li>  The file will address the explore some hypothesis on the data</li> 
<li>What are the null and alternate hypotheses?</li>
<li>Compute margin of error, confidence interval, and p-value.</li>
<li>Write a story describing the statistical significance in the context or the original problem.</li>
<li> Explore following data and hypothesis : 
    <ul>
     <li>Exploration 1 :Gender for loan approve and rejection</li>
    </ul>
</li>        
</ol>
</div>

In [2]:
pd.set_option('display.max_colwidth', -1)
color_success='#00ff66'#kickstarter green#'#2BDE73'
color_fail='#FF192F'#'#FFC7CE'#'#FFC7CE'#'#FF0000' # red
stateSuccess='Successful'
stateFail='Failed'
# the statistical significance value is 5%
siginificane_level =0.005
# for 95% the critcal z value is 1.96 ( based on z table)
critical_value =1.96
noofsample=100

In [3]:
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }'
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='df_selectdata_hmda_cenus.sa'
df_selectdata = pickle.load( open( pickle_file, "rb" ) )
#df_final_months.info()
#df_final_months.head()

In [4]:
df_selectdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 21 columns):
Year                    int64
PropertyType            int64
LoanPurpose             float64
Occupancy               float64
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      float64
CoApplicantEthnicity    float64
ApplicantRace           float64
CoApplicantRace         float64
ApplicantSex            float64
CoApplicantSex          float64
ApplicantIncome         float64
PurchaserType           float64
StateName               object
CountyName              object
CLASSCODEFIPS           object
Result                  int64
dtypes: float64(15), int64(3), object(3)
memory usage: 5.2+ GB


In [5]:
df_selectdata.Result.unique()

array([0, 1], dtype=int64)

##  Analysis start

<div class="span5 alert alert-info">
<h4>
Exploration 1 :Action Type for approve and and rejected loans</h4>
</div>

In [6]:
analyzecol='Result'
dfHypo=df_selectdata[df_selectdata.ApplicantSex<=2].loc[:,['ApplicantSex',analyzecol]]

In [7]:
# number of approve for male
sum(dfHypo[dfHypo.ApplicantSex==1].Result)

16080586

In [8]:
# number of approve for female
sum(dfHypo[dfHypo.ApplicantSex==2].Result)

6843921

In [9]:
dfHypo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28794250 entries, 0 to 49681
Data columns (total 2 columns):
ApplicantSex    float64
Result          int64
dtypes: float64(1), int64(1)
memory usage: 659.0 MB


In [10]:
dfHypo.ApplicantSex.unique()

array([1., 2.])

In [11]:
dfHypo.describe()

,ApplicantSex,Result
count,2.879425e+07,2.879425e+07
mean,1.308363e+00,7.961488e-01
std,4.618173e-01,4.028597e-01
min,1.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00
75%,2.000000e+00,1.000000e+00
max,2.000000e+00,1.000000e+00


#### Get the dataframe

In [12]:
dfHypo_male=dfHypo[dfHypo.ApplicantSex==1]
dfHypo_female=dfHypo[dfHypo.ApplicantSex==2]

In [13]:
dfHypo_male.Result.describe()

count    1.991517e+07
mean     8.074540e-01
std      3.942995e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: Result, dtype: float64

In [14]:
dfHypo_female.Result.describe()

count    8.879076e+06
mean     7.707920e-01
std      4.203233e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: Result, dtype: float64

In [15]:
mean_male_Result=dfHypo_male.Result.mean()
mean_female_Result=dfHypo_female.Result.mean()
mean_diff_Result=mean_male_Result-mean_female_Result
mean_diff_Result


0.03666192609733088

<div class="span5 alert alert-info" style="background-color:#ffff66; color:black">

There is difference between the mean of Result between male and female loans. We can further analyze this  hypothesis. <b>Answer :</b> The hypothesis are as follows
    <br/>
    <b> Null Hypothesis : </b> There is <b>no</b> difference in result in male or female approved loan. Which <i>means for result for men - means for result for female</i> equals <b>Zero</b> .
    <br/>
    <b> Alternate Hypothesis :  </b> There is <b>siginifcant</b> difference in result in male or female approved loan. Which <i>means for result for men - means for result for female</i> not equals <b>Zero</b> .
</div>

In [17]:
mean_male_calls, sd_male_calls, var_male_calls= dfHypo_male.Result.mean(), dfHypo_male.Result.std(), dfHypo_male.Result.var()
mean_female_calls, sd_female_calls, var_female_calls= dfHypo_female.Result.mean(), dfHypo_female.Result.std(), dfHypo_female.Result.var()

print (' The mean for calls for male candidate is ' + str(mean_male_calls))
print (' The standard deviation for calls for male candidate is ' + str(sd_male_calls))
print (' The varience for calls for male candidate is ' + str(var_male_calls))
print (' The mean for calls for female candidate is ' + str(mean_female_calls))
print (' The standard deviation for calls for female candidate is ' + str(sd_female_calls))
print (' The varience for calls for female candidate is ' + str(var_female_calls))

 The mean for calls for male candidate is 0.8074539544570386
 The standard deviation for calls for male candidate is 0.3942994720520549
 The varience for calls for male candidate is 0.1554720736605292
 The mean for calls for female candidate is 0.7707920283597077
 The standard deviation for calls for female candidate is 0.42032332468866307
 The varience for calls for female candidate is 0.17667169727733129


In [18]:
difference_population_mean =mean_male_calls-mean_female_calls
difference_population_mean

0.03666192609733088

###  Variance for all calls based on bernoulli distribution is sum of  Variance (male) loans and Variance(female) loans

In [19]:
# proporation calculation for colored people
total_female=len(dfHypo_female)# total no of people of color
female_got_approved=len(dfHypo_female[dfHypo_female.Result==1]) # female got Result 1
proporation_female_approve=female_got_approved/total_female # population proporation of female who got Result 1
print('The total female with Result approved in population '+str(female_got_approved))
print('The total female in population '+str(total_female))
print('The colored people Result /  total female '+str(proporation_female_approve))

# proporation calculation for non colored people
total_male=len(dfHypo_male)# total no of people of non color
male_got_approved=len(dfHypo_male[dfHypo_male.Result==1])# male got Result 1
proporation_male_approve=male_got_approved/total_male # population proporation of male who got Result 1
print('The total male with Result in population '+str(male_got_approved))
print('The total male in population '+str(total_male))
print('The male Result / total male '+str(proporation_male_approve))

# calculate variance of color people
#P(1-p)/N
variance_female=(proporation_female_approve*(1-proporation_female_approve))/total_female
print('The female variance %.12f' % float(variance_female))

# calculate variance of non color people
variance_male=(proporation_male_approve*(1-proporation_male_approve))/total_male
print('The male people variance %.12f' % float(variance_male))

population_variance=variance_male+variance_female

print('The total population varience %.12f' % (float(population_variance)))

The total female with Result approved in population 6843921
The total female in population 8879076
The colored people Result /  total female 0.7707920283597077
The total male with Result in population 16080586
The total male in population 19915174
The male Result / total male 0.8074539544570386
The female variance 0.000000019898
The male people variance 0.000000007807
The total population varience 0.000000027704


###  standard deviation for all calls square root of total varaience

In [20]:
# standard devaiation for total population
population_standard_deviation=math.sqrt(population_variance)
print('The total population standard deviation '+str(population_standard_deviation))

The total population standard deviation 0.00016644591762137129


### 95 % chance which is critical value(1.96) that poupulaton mean (true mean) is within  difference of population  mean (0.032) will be Confidence interval

In [21]:
# 95 % chance for confidence interval 
conf_interval= critical_value*population_standard_deviation

print('The confidence interval with 95 % '+str(conf_interval))

The confidence interval with 95 % 0.00032623399853788773


In [24]:
# margin of erro 
#population_standard_deviation-conf_interval , population_standard_deviation+conf_interval
difference_population_mean+np.array([-1, 1]) * conf_interval

array([0.03633569, 0.03698816])

### Based on null hypothesis, population proporation of women who got loan approved  -population proporation of male who got loan approved is zero. So difference_population_mean (0.032) away from 0 . We get z score for this 
#### To calculate the z score we need standard deviation with null hypothesis. And based on null hypothesis both proporation are equal and so proproartion of woman got loan equals pepole of men got loan

In [25]:
# null hypoths,p1=p2
# z score for 
null_hypothese_proporation=(male_got_approved+female_got_approved)/(total_male+total_female)  #(157+235)/4870
#new sd based on hypo= math.sqrt(2p*(1-p)/n)
standard_deviation_z=math.sqrt((2*null_hypothese_proporation*(1-null_hypothese_proporation))/(2435))
print(standard_deviation_z)

0.011545673973341627


In [27]:
#t=difference_population_mean-0/(new sd based on hypo)
z_value = (difference_population_mean-0)/standard_deviation_z
print (z_value)

3.1753820679486875


<div class="span5 alert alert-info" style='background-color:#ffff66; color:black'><b>Answer :</b> The p- value for z number is   <b>0.0115456739733416274 </b></div>

In [28]:
if (z_value>critical_value):
    print ('The calculated z-score is higher than critical z value hence reject the null hypothesis')
else:
    print('Null hypotheses is true')

The calculated z-score is higher than critical z value hence reject the null hypothesis


<div class="span5 alert alert-info" style='background-color:#ffff66; color:black'><b>Answer :</b> The calculated z-score is higher than critical z value hence will allow to <b>reject </b>the null hypothesis.</div>

#### Calculate Z score and p score for this 

In [29]:
zstat, pvalue = wstats.ztest(dfHypo_male[analyzecol], dfHypo_female[analyzecol], alternative='two-sided',
                    value=0, usevar='pooled', ddof=1.0)
print ('The z stat is {} and p value is {}'.format(zstat,pvalue))

The z stat is 225.7191608052462 and p value is 0.0


In [30]:
zstat, pvalue = wstats.ztest(dfHypo_male[analyzecol], dfHypo_female[analyzecol], alternative='larger',#'smaller',
                    value=0, usevar='pooled', ddof=1.0)
print ('The z stat is {} and p value is {}'.format(zstat,pvalue))

The z stat is 225.7191608052462 and p value is 0.0


####  Calculate T score and p value to test same hypothesis

In [31]:
# T-score
tstat, p_from_t = stats.ttest_ind(dfHypo_male[analyzecol], dfHypo_female[analyzecol], equal_var=False)
print('t-statistic: ', tstat)
print('p-value: ', p_from_t)

t-statistic:  220.26328358859774
p-value:  0.0


In [32]:
if (p_from_t<siginificane_level):
    print ('The calulcated p value is lower then 0.05 % significant value will allow to reject the null hypothesis')
else:
    print('Null hypotheses is true')

The calulcated p value is lower then 0.05 % significant value will allow to reject the null hypothesis


<div class="span5 alert alert-info" style='background-color:#ffff66; color:black'><b>Answer :</b> The calulcated p value is lower then 0.05 % significant value will allow to <b>reject </b>the null hypothesis.</div>

<div class="span5 alert alert-info analysis" >
<h4>
Conclusion : Exploration 1 :Gender for loan approve and rejection</h4>
    <ul>
     <li> There is enough evidence to reject Null hypothesis </li> 
     <li> The conculsion is that loan approval for male and female vaires </li> 
    </ul>
</div>